In [1]:
import pandas as pd
import csv
import datetime
import re

crsp_location = '../archive/CRSP_small_database.csv'
breach_location = '../data/data_breaches_final.csv'
ff_location = '../data/FF_factors.csv'
trends_tic_location = '../data/trends_tic.csv'
trends_conm_location = '../data/trends_conm.csv'
out_location = '../data/CRSP_merged_feb.csv'

In [2]:
remove = ['CORP', 'INC', 'LTD', 'CORPORATION', 'INCORPORATED', 'LLC', 'LTD', 'GROUP', 'NV', 'GRP', 'PLC']

def clean_name(name):
    name = str(name)
    name = re.sub(r'[^\s\w]+', '', name)
    namewords = name.split()
    resultwords = [word for word in namewords if word not in remove]
    result = ' '.join(resultwords)
    result = result.strip(' ')
    return result

In [3]:
crsp = pd.read_csv(crsp_location)
crsp['date'] = pd.to_datetime(crsp['date'], format='%Y%m%d')
crsp.columns
crsp['datamonth'] = crsp['date'].apply(lambda x: x.replace(day = 1))
crsp['clean_name'] = crsp['COMNAM'].apply(clean_name)
#crsp['clean_tic'] = crsp['TICKER'].apply(clean_tic)

/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,8,15,19,21,26,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
breach = pd.read_csv(breach_location)
breach['date'] = pd.to_datetime(breach['Date Made Public'], format='%B %d, %Y')
breach['TICKER'] = breach['tic']
breach = breach[breach['match'] == 1]

In [5]:
ff = pd.read_csv(ff_location)
ff['date'] = pd.to_datetime(ff['date'], format='%Y%m%d')

In [9]:
# trends_company = pd.read_csv(trends_conm_location, index_col='date').stack()
# trends_company = trends_company.reset_index()
# trends_company['date'] = pd.to_datetime(trends_company['date'])
# trends_company.columns = ['datamonth', 'clean_name', 'trend_index_company']

# trends_tic = pd.read_csv(trends_tic_location, index_col='date').stack()
# trends_tic = trends_tic.reset_index()
# trends_tic['date'] = pd.to_datetime(trends_tic['date'])
# trends_tic.columns = ['datamonth', 'clean_tic', 'trend_index_tic']

In [6]:
out = pd.merge(crsp, breach, how='outer', on=['date', 'TICKER'])


In [19]:
out = pd.merge(out, ff, how='left', on='date')


NameError: name 'close' is not defined

In [12]:
# out = pd.merge(out, trends_company, how='outer', on=['datamonth','clean_name'])
# out = pd.merge(out, trends_tic, how='outer', on=['datamonth','clean_tic'])

In [21]:
del breach
del ff


NameError: name 'breach' is not defined

In [28]:
crsp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161319 entries, 0 to 1161318
Data columns (total 40 columns):
PERMNO        1161319 non-null int64
date          1161319 non-null datetime64[ns]
SHRCD         1161318 non-null float64
EXCHCD        1161318 non-null float64
SICCD         1161318 non-null float64
NCUSIP        1161318 non-null object
TICKER        1149470 non-null object
COMNAM        1161318 non-null object
SHRCLS        117751 non-null object
TSYMBOL       1149470 non-null object
PRIMEXCH      1161318 non-null object
TRDSTAT       1161318 non-null object
SECSTAT       1161318 non-null object
PERMCO        1161319 non-null int64
HEXCD         1161319 non-null int64
CUSIP         1161319 non-null object
DLAMT         103 non-null float64
DLPDT         103 non-null float64
DLSTCD        103 non-null float64
DLRETX        103 non-null object
DLPRC         103 non-null float64
DLRET         103 non-null object
BIDLO         1149263 non-null float64
ASKHI         1149263 non

In [18]:
#out[(out['match'] == 1) & out['COMNAM'].isnull()]
pd.merge(out, crsp, how='left', on=['TICKER'], suffixes=('_x', False))

MemoryError: 

In [14]:
out.to_csv(out_location, index=False, quoting=csv.QUOTE_ALL)